# 以 MNIST 資料集測試遷移學習
學習前五種數字 (0,1,2,3,4) 的特徵，轉移到後五種數字 (5,6,7,8,9) 的分類辨識。

In [25]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical

# 指定亂數種子
seed = 7
np.random.seed(seed)

# 載入資料集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 建立2個資料集，一個數字小於 5，一個數字大於等於 5
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

# 將圖片轉換成 4D 張量
x_train_lt5 = x_train_lt5.reshape((x_train_lt5.shape[0], 28, 28, 1)).astype('float32')
x_test_lt5 = x_test_lt5.reshape((x_test_lt5.shape[0], 28, 28, 1)).astype('float32')
x_train_gte5 = x_train_gte5.reshape((x_train_gte5.shape[0], 28, 28, 1)).astype('float32')
x_test_gte5 = x_test_gte5.reshape((x_test_gte5.shape[0], 28, 28, 1)).astype('float32')

# 因為是固定範圍, 所以執行正規化, 從 0-255 至 0-1
x_train_lt5 = x_train_lt5 / 255
x_test_lt5 = x_test_lt5 / 255
x_train_gte5 = x_train_gte5 / 255
x_test_gte5 = x_test_gte5 / 255

# One-hot編碼
y_train_lt5 = to_categorical(y_train_lt5, 5)
y_test_lt5 = to_categorical(y_test_lt5, 5)
y_train_gte5 = to_categorical(y_train_gte5, 5)
y_test_gte5 = to_categorical(y_test_gte5, 5)

# 定義模型
model = Sequential()
model.add(Conv2D(8, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(8, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

model.summary()   # 顯示模型摘要資訊


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 8)         80        
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 8)        0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 11, 11, 8)         584       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 5, 5, 8)          0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 200)               0         
                                                                 
 dense_5 (Dense)             (None, 64)               

In [26]:
print(x_train.shape)
print(x_train_lt5.shape)
print(x_train_gte5.shape)

print(y_train.shape)
print(y_train_lt5.shape)
print(y_train_gte5.shape)

print(x_test.shape)
print(x_test_lt5.shape)
print(x_test_gte5.shape)

print(y_test.shape)
print(y_test_lt5.shape)
print(y_test_gte5.shape)


(60000, 28, 28)
(30596, 28, 28, 1)
(29404, 28, 28, 1)
(60000,)
(30596, 5)
(29404, 5)
(10000, 28, 28)
(5139, 28, 28, 1)
(4861, 28, 28, 1)
(10000,)
(5139, 5)
(4861, 5)


In [27]:
# 編譯模型
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練模型
history = model.fit(x_train_lt5, y_train_lt5, validation_split=0.2, epochs=10, batch_size=128, verbose=2)


Epoch 1/10
192/192 - 8s - loss: 0.3795 - accuracy: 0.8777 - val_loss: 0.0900 - val_accuracy: 0.9706 - 8s/epoch - 41ms/step
Epoch 2/10
192/192 - 7s - loss: 0.0986 - accuracy: 0.9712 - val_loss: 0.0532 - val_accuracy: 0.9851 - 7s/epoch - 36ms/step
Epoch 3/10
192/192 - 7s - loss: 0.0646 - accuracy: 0.9806 - val_loss: 0.0392 - val_accuracy: 0.9879 - 7s/epoch - 36ms/step
Epoch 4/10
192/192 - 7s - loss: 0.0504 - accuracy: 0.9841 - val_loss: 0.0347 - val_accuracy: 0.9894 - 7s/epoch - 36ms/step
Epoch 5/10
192/192 - 7s - loss: 0.0416 - accuracy: 0.9867 - val_loss: 0.0318 - val_accuracy: 0.9904 - 7s/epoch - 36ms/step
Epoch 6/10
192/192 - 7s - loss: 0.0351 - accuracy: 0.9891 - val_loss: 0.0269 - val_accuracy: 0.9918 - 7s/epoch - 36ms/step
Epoch 7/10
192/192 - 7s - loss: 0.0312 - accuracy: 0.9910 - val_loss: 0.0244 - val_accuracy: 0.9926 - 7s/epoch - 36ms/step
Epoch 8/10
192/192 - 7s - loss: 0.0262 - accuracy: 0.9916 - val_loss: 0.0236 - val_accuracy: 0.9922 - 7s/epoch - 36ms/step
Epoch 9/10
192/1

In [28]:
# 評估模型 (lt5)
loss, accuracy = model.evaluate(x_test_lt5, y_test_lt5, verbose=0)
print('測試資料集(lt5)的準確度 = {:.2f}'.format(accuracy))


測試資料集(lt5)的準確度 = 1.00


In [29]:
# 評估模型 (gte5)
loss, accuracy = model.evaluate(x_test_gte5, y_test_gte5, verbose=0)
print('測試資料集(gte5)的準確度 = {:.2f}'.format(accuracy))


測試資料集(gte5)的準確度 = 0.41


In [30]:
# 顯示各神經層
print(len(model.layers))
for i in range(len(model.layers)):
    print(i, model.layers[i])

# 凍結上層模型
for i in range(4):
    model.layers[i].trainable = False


8
0 <keras.layers.convolutional.Conv2D object at 0x7f41c1eb4710>
1 <keras.layers.pooling.MaxPooling2D object at 0x7f41c6157990>
2 <keras.layers.convolutional.Conv2D object at 0x7f41bfd48ad0>
3 <keras.layers.pooling.MaxPooling2D object at 0x7f41bfd48190>
4 <keras.layers.core.flatten.Flatten object at 0x7f41c6548e90>
5 <keras.layers.core.dense.Dense object at 0x7f41bfda9250>
6 <keras.layers.core.dropout.Dropout object at 0x7f41bfcb9a50>
7 <keras.layers.core.dense.Dense object at 0x7f41bfcd1290>


In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 8)         80        
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 8)        0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 11, 11, 8)         584       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 5, 5, 8)          0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 200)               0         
                                                                 
 dense_5 (Dense)             (None, 64)               

In [32]:
# 編譯模型
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練模型
history = model.fit(x_train_gte5, y_train_gte5, validation_split=0.2, epochs=5, batch_size=128, verbose=2)

# 評估模型
loss, accuracy = model.evaluate(x_test_gte5, y_test_gte5, verbose=0)
print('測試資料集的準確度 = {:.2f}'.format(accuracy))


Epoch 1/5
184/184 - 4s - loss: 0.6511 - accuracy: 0.8020 - val_loss: 0.1530 - val_accuracy: 0.9553 - 4s/epoch - 22ms/step
Epoch 2/5
184/184 - 3s - loss: 0.1980 - accuracy: 0.9381 - val_loss: 0.1044 - val_accuracy: 0.9679 - 3s/epoch - 18ms/step
Epoch 3/5
184/184 - 3s - loss: 0.1452 - accuracy: 0.9552 - val_loss: 0.0823 - val_accuracy: 0.9750 - 3s/epoch - 18ms/step
Epoch 4/5
184/184 - 3s - loss: 0.1181 - accuracy: 0.9632 - val_loss: 0.0737 - val_accuracy: 0.9762 - 3s/epoch - 17ms/step
Epoch 5/5
184/184 - 3s - loss: 0.1037 - accuracy: 0.9685 - val_loss: 0.0674 - val_accuracy: 0.9811 - 3s/epoch - 17ms/step
測試資料集的準確度 = 0.98


## 如果也要同時保留原始資料的分類效果?! (有難度)

In [33]:
print(x_train.shape)
print(x_train_lt5.shape)
print(x_train_gte5.shape)

print(y_train.shape)
print(y_train_lt5.shape)
print(y_train_gte5.shape)

print(x_test.shape)
print(x_test_lt5.shape)
print(x_test_gte5.shape)

print(y_test.shape)
print(y_test_lt5.shape)
print(y_test_gte5.shape)


(60000, 28, 28)
(30596, 28, 28, 1)
(29404, 28, 28, 1)
(60000,)
(30596, 5)
(29404, 5)
(10000, 28, 28)
(5139, 28, 28, 1)
(4861, 28, 28, 1)
(10000,)
(5139, 5)
(4861, 5)


In [34]:
model.pop()
model.add(Dense(10, activation='softmax', name='10_classes'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 8)         80        
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 8)        0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 11, 11, 8)         584       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 5, 5, 8)          0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 200)               0         
                                                                 
 dense_5 (Dense)             (None, 64)               

In [35]:
# 複習 np.pad 用法
a=[[1,2],[3,4]]
out = np.pad(a, ((0,0),(3,0)), mode='constant', constant_values=0) # (2,3)表示左側補2個，右側補3個
print(a)
print(out)

[[1, 2], [3, 4]]
[[0 0 0 1 2]
 [0 0 0 3 4]]


In [36]:
# y_test 前方補0
y_test_lt5_onehot = np.pad(y_test_lt5, ((0,0),(5,0)), mode='constant', constant_values=0)
y_test_gte5_onehot = np.pad(y_test_gte5, ((0,0),(5,0)), mode='constant', constant_values=0)

print(y_test_gte5.shape)
print(y_test_gte5_onehot.shape)
print(y_test_gte5_onehot)

(4861, 5)
(4861, 10)
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [37]:
# 編譯模型
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練模型
history = model.fit(x_test_gte5, y_test_gte5_onehot, validation_split=0.2, epochs=5, batch_size=128, verbose=2)


Epoch 1/5
31/31 - 1s - loss: 1.7424 - accuracy: 0.4262 - val_loss: 0.4564 - val_accuracy: 0.9291 - 1s/epoch - 40ms/step
Epoch 2/5
31/31 - 1s - loss: 0.5757 - accuracy: 0.8102 - val_loss: 0.1735 - val_accuracy: 0.9702 - 580ms/epoch - 19ms/step
Epoch 3/5
31/31 - 1s - loss: 0.3515 - accuracy: 0.8794 - val_loss: 0.1175 - val_accuracy: 0.9753 - 558ms/epoch - 18ms/step
Epoch 4/5
31/31 - 1s - loss: 0.2759 - accuracy: 0.9144 - val_loss: 0.0897 - val_accuracy: 0.9815 - 557ms/epoch - 18ms/step
Epoch 5/5
31/31 - 1s - loss: 0.2307 - accuracy: 0.9259 - val_loss: 0.0777 - val_accuracy: 0.9794 - 574ms/epoch - 19ms/step


In [38]:
# 評估模型 (lt5)
loss, accuracy = model.evaluate(x_test_lt5, y_test_lt5_onehot, verbose=0)
print('測試資料集(lt5)的準確度 = {:.2f}'.format(accuracy))

# 評估模型 (gte5)
loss, accuracy = model.evaluate(x_test_gte5, y_test_gte5_onehot, verbose=0)
print('測試資料集(gte5)的準確度 = {:.2f}'.format(accuracy))


測試資料集(lt5)的準確度 = 0.39
測試資料集(gte5)的準確度 = 0.97
